In [1]:
import numpy as np
import cv2
import torch 
import torchvision 
from engine import train_one_epoch, evaluate
from data_reading import WindowDataset
import utils
from torch.utils.data import DataLoader
import itertools
from torch.utils.tensorboard import SummaryWriter


In [2]:
def get_model():
    # Initialize Model, from pytorch tutorial: https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)
    num_classes = 2 # window + background
    in_features = model.roi_heads.box_predictor.cls_score.in_features # Number of input features for the classifier
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
    return model.double()

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'running on device {device}')
model = get_model()
model.to(device)

running on device cuda


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [4]:
params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(params, lr = 0.00001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma = 0.1)

In [5]:
dataset_train = WindowDataset()
dataset_val = WindowDataset(images_folder="data/val/images/", labels_path="data/val/anno")
dataset_test = WindowDataset(images_folder="data/test/images/", labels_path="data/test/anno")

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [6]:
train = DataLoader(dataset_train, batch_size=1, shuffle=True, collate_fn=utils.collate_fn)
print(next(iter(train)))

val = DataLoader(dataset_val, batch_size=1, shuffle=True, collate_fn=utils.collate_fn)
test = DataLoader(dataset_test, batch_size=1, shuffle=True, collate_fn=utils.collate_fn)

((tensor([[[0.0824, 0.0706, 0.0627,  ..., 0.1451, 0.1412, 0.1843],
         [0.0392, 0.0510, 0.0784,  ..., 0.1451, 0.1451, 0.1922],
         [0.0471, 0.0431, 0.0588,  ..., 0.1412, 0.1373, 0.1843],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.2196, 0.2392, 0.2667],
         [0.0000, 0.0000, 0.0000,  ..., 0.2235, 0.2431, 0.2667],
         [0.0000, 0.0000, 0.0000,  ..., 0.2275, 0.2392, 0.2627]],

        [[0.1529, 0.1412, 0.1294,  ..., 0.1647, 0.1608, 0.2039],
         [0.1098, 0.1216, 0.1412,  ..., 0.1647, 0.1647, 0.2118],
         [0.1098, 0.1059, 0.1216,  ..., 0.1608, 0.1569, 0.2039],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.2627, 0.2824, 0.3098],
         [0.0000, 0.0000, 0.0000,  ..., 0.2667, 0.2863, 0.3098],
         [0.0000, 0.0000, 0.0000,  ..., 0.2627, 0.2824, 0.3059]],

        [[0.1059, 0.0941, 0.0902,  ..., 0.1412, 0.1373, 0.1804],
         [0.0627, 0.0745, 0.1020,  ..., 0.1412, 0.1412, 0.1882],
         [0.0706, 0.0667, 0.0824,  ..., 0.1333, 0.1294, 

In [7]:
print(next(iter(train))[1][0]['labels'].dtype)

torch.int64


In [ ]:
print(device)
evaluate(model, train, device, print_freq=100)

In [8]:
num_epochs = 16
writer = SummaryWriter()
for epoch in range(num_epochs):
    # Train for one epoch, while printing every 10 iterations
    train_one_epoch(writer, model, optimizer, train, device, epoch, print_freq=100)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, val, device, print_freq=100)
    torch.save(model.state_dict(),'baseline10_8.weights')

print("DONE")

Epoch: [0]  [  0/612]  eta: 1:59:40  lr: 0.000010  loss: 1.3118 (1.3118)  loss_classifier: 0.6276 (0.6276)  loss_box_reg: 0.3877 (0.3877)  loss_objectness: 0.2787 (0.2787)  loss_rpn_box_reg: 0.0178 (0.0178)  time: 11.7323  data: 0.0050  max mem: 2046
Epoch: [0]  [100/612]  eta: 1:32:36  lr: 0.000010  loss: 0.1451 (0.3348)  loss_classifier: 0.0413 (0.1196)  loss_box_reg: 0.0558 (0.1293)  loss_objectness: 0.0058 (0.0570)  loss_rpn_box_reg: 0.0193 (0.0289)  time: 10.8438  data: 0.0012  max mem: 2888
Epoch: [0]  [200/612]  eta: 1:14:29  lr: 0.000010  loss: 0.1028 (0.2758)  loss_classifier: 0.0293 (0.0963)  loss_box_reg: 0.0466 (0.1222)  loss_objectness: 0.0021 (0.0339)  loss_rpn_box_reg: 0.0137 (0.0234)  time: 10.8464  data: 0.0011  max mem: 2888
Epoch: [0]  [300/612]  eta: 0:56:04  lr: 0.000010  loss: 0.1014 (0.2560)  loss_classifier: 0.0260 (0.0859)  loss_box_reg: 0.0512 (0.1186)  loss_objectness: 0.0033 (0.0288)  loss_rpn_box_reg: 0.0149 (0.0228)  time: 10.6040  data: 0.0011  max mem: 2

In [ ]:
model.eval()
img, labs = next(iter(train))
print(labs)
print(img[0].shape)
img = tuple([i.to(device) for i in img])


In [ ]:
model(img)